In [2]:
%%javascript
//IPython.OutputArea.auto_scroll_threshold = 9999;   //设置输出>9999时才出现滑动窗口
IPython.OutputArea.prototype._should_scroll = function(){return false}  // 设置不出现滑动窗口 true, auto, false

<IPython.core.display.Javascript object>

In [3]:
import pandas as pd
pd.options.display.max_columns = None
pd.options.display.max_rows = None
df_trade = pd.read_csv('wangsheng_stoploss_v8_rank_simple1_trade.csv')
df_long_signal_basis = pd.read_csv('wangsheng_stoploss_v8_rank_simple1_long_signal_basis.csv')
df_long_stop_basis = pd.read_csv('wangsheng_stoploss_v8_rank_simple1_long_stop_basis.csv')

ids = list(set(df_trade['name'].values.tolist()))
ids = [str(id) for id in ids]



In [6]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from ipywidgets import interact
from redis_common import redis_object
from datetime import datetime
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
from IPython.display import display, Markdown, HTML


def indicators(symbol):
    
    x = int(symbol)
    
    kdata = pd.read_csv('wangsheng_stoploss_v8_rank_simple1_close.csv')
    
    df = pd.read_csv('wangsheng_stoploss_v8_rank_simple1_indicators_day.csv')
    df1 = df.loc[df['name']==x].copy()
    df1.loc[:,'time'] = df1['time'].astype('datetime64').copy()
    df1 = df1.sort_values('time')
    
    # trade table
    df_t1 = df_trade.loc[df_trade['name']==int(symbol),['name', 'open_date', 'close_date', 'buy_price', 'sell_price', 'pnl']]
    df_t1['open_date'] = df_t1['open_date'].astype('datetime64').copy()
    df_t1['close_date'] = df_t1['close_date'].astype('datetime64').copy()

    df_t1 = df_t1.sort_values('open_date')
    
    df_t1 = df_t1.reset_index()
    display(Markdown('## trade table'))
    display(HTML(df_t1[['name','open_date', 'buy_price','close_date',  'sell_price', 'pnl']].to_html()))
    
    # long_signal_basis table
    df_t2 = df_long_signal_basis.loc[df_long_signal_basis['name']==int(symbol),['name','time','close','local_min_point']]
    df_t2 = df_t2.drop_duplicates(subset=['name','time','close','local_min_point'], keep='last')

    df_t2['time'] = df_t2['time'].astype('datetime64').copy()

    df_t2 = df_t2.sort_values('time')
    
    df_t2 = df_t2.reset_index()
    display(Markdown('## long signal basis'))
    display(HTML(df_t2[['name','time','close','local_min_point']].to_html()))
    
    # long_stop_basis table
    df_t3 = df_long_stop_basis.loc[df_long_stop_basis['name']==int(symbol),['name','time','last_buy_price','stop_loss_percent','close','highest_price','matr','matr_multi']]
    df_t3 = df_t3.drop_duplicates(subset=['name','time','last_buy_price','stop_loss_percent','close','highest_price','matr','matr_multi'], keep='last')

    df_t3['time'] = df_t3['time'].astype('datetime64').copy()

    df_t3 = df_t3.sort_values('time')
    
    df_t3 = df_t3.reset_index()
    display(Markdown('## long stop basis'))
    display(df_t3.drop(['index'],axis=1))
    
    
#     fig0 = ff.create_table(df_signal[['name','open_date', 'buy_price','close_date',  'sell_price', 'pnl']])
#     fig0.show()
    
    columns= list(df1.columns)
    columns.remove('name')
    
#     fig = go.Figure()
#     fig.add_trace(go.Scatter(x=kdata['time'], y=kdata['close'],
#                              xperiod=1000*60,line_shape='hv',
#                              connectgaps=True,
#                             mode='lines',
#                             name='close'))
#     fig.add_trace(go.Scatter(x=df1['time'], 
#                              y=df1['extrema_max'],
#                              mode='lines',line_shape='vh',
#                              name='columns'
#                             ))

#     fig.show()
    display(Markdown('## signal detail'))
    fig = make_subplots(
        rows=3, cols=1,
        shared_xaxes=True,
        vertical_spacing=0.03,
        specs=[[{"type": "scatter"}],
               [{"type": "scatter"}],
               [{"type": "scatter"}],
              ]
    )
    
    fig.add_trace(
        go.Scatter(
            x=kdata['time'], 
            y=kdata['close'],
            xperiod=1000*60,line_shape='hv',
            connectgaps=True,
            mode='lines',
            name='close'
        ),
        row=1, col=1
    )
    """
    fig.add_trace(
        go.Scatter(
            x=df1['time'], 
            y=df1['extrema_max'],
            mode='lines',
            line_shape='vh',
            name='extrema_max'
        ),
        row=1, col=1
    )
    """
    
    
    fig.add_trace(
        go.Scatter(
            x=df1['time'], 
            y=df1['extrema_min'],
            mode='lines',
            line_shape='vh',
            name='extrema_min'
        ),
        row=1, col=1
    )
    
    fig.add_trace(
        go.Scatter(
            x=df1['time'], 
            y=df1['day_highest'],
            mode='lines',
            line_shape='vh',
            name='day_highest'
        ),
        row=1, col=1
    )
    
    fig.add_trace(
        go.Scatter(
            x=df1['time'], 
            y=df1['avg_close'],
            mode='lines',
            line_shape='vh',
            name='avg_close'
        ),
        row=1, col=1
    )
    
    # 添加买卖信号
    fig.add_trace(
        go.Scatter(
            x=df_t1['open_date'], 
            y=df_t1['buy_price'],
            mode='markers',
            name='buy_signal',
            marker={'symbol':'triangle-up', 'size': 12}
        ),
        row=1, col=1
    )
    fig.add_trace(
        go.Scatter(
            x=df_t1['close_date'], 
            y=df_t1['sell_price'],
            mode='markers',
            name='sell_signal',
            marker={'symbol':'triangle-down', 'size': 12}
        ),
        row=1, col=1
    )
    
    
    fig.add_trace(
        go.Scatter(
            x=df1['time'], 
            y=df1['point_period'],
            mode='lines',
            line_shape='vh',
            name='point_period'
        ),
        row=2, col=1
    )
    
    # thirds_period
    fig.add_trace(
        go.Scatter(
            x=df1['time'], 
            y=df1['thirds_period'],
            mode='lines',
            line_shape='vh',
            name='thirds_period'
        ),
        row=2, col=1
    )
    
    fig.add_trace(
        go.Scatter(
            x=df1['time'], 
            y=df1['matr'],
            mode='lines',
            line_shape='vh',
            name='matr'
        ),
        row=3, col=1
    )
    fig.add_trace(
        go.Scatter(
            x=df1['time'], 
            y=df1['init_atr_multi'],
            mode='lines',
            line_shape='vh',
            name='init_atr_multi'
        ),
        row=3, col=1
    )
    
    
    # base_percent
    fig.add_trace(
        go.Scatter(
            x=df1['time'], 
            y=df1['base_percent'],
            mode='lines',
            line_shape='vh',
            name='base_percent'
        ),
        row=3, col=1
    )
    
    """
    fig.add_trace(
        go.Scatter(
            x=kdata['time'], 
            y=kdata['volume'],
            xperiod=1000*60,line_shape='hv',
            connectgaps=True,
            mode='lines',
            name='volume'
        ),
        row=4, col=1
    )
    
    fig.add_trace(
        go.Scatter(
            x=df1['time'], 
            y=df1['avg_volume'],
            mode='lines',
            line_shape='vh',
            name='avg_volume'
        ),
        row=4, col=1
    )
    """
    
    

#     fig.add_trace(
#         go.Table(
#             header=dict(
#                 values=["Date", "Number<br>Transactions", "Output<br>Volume (BTC)",
#                         "Market<br>Price", "Hash<br>Rate", "Cost per<br>trans-USD",
#                         "Mining<br>Revenue-USD", "Trasaction<br>fees-BTC"],
#                 font=dict(size=10),
#                 align="left"
#             ),
#             cells=dict(
#                 values=[df[k].tolist() for k in df.columns[1:]],
#                 align = "left")
#         ),
#         row=1, col=1
#     )
    fig.update_layout(
        height=1200,
        showlegend=True,
#         title_text="signal detail",
    )
#     fig.update_xaxes(rangeslider_visible=True)
    fig.show()

interact(indicators, symbol=ids);

interactive(children=(Dropdown(description='symbol', options=('913256450', '913323524', '913256965', '91332403…